# HumanEval Evaluation: Qwen2.5-Coder-1.5B-Instruct (Error-Aware Prompt)

This notebook evaluates `Qwen/Qwen2.5-Coder-1.5B-Instruct` on HumanEval using pass@1 metric.

**This version uses an ERROR-AWARE PROMPT** designed to prevent syntax errors.

**Features:**
- Calculate pass@1 on all 164 HumanEval problems
- Log prompts and generated outputs to a separate txt file
- Calculate error percentages: syntax errors, runtime errors, incomplete code errors
- Track which problem numbers execute successfully

## 1. Setup & Dependencies

In [1]:
# Install required packages
!pip install -q transformers accelerate bitsandbytes torch
!pip install -q human-eval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour i

In [2]:
import json
import os
import re
import ast
import traceback
from typing import Dict, List, Tuple, Any
from collections import defaultdict
import signal
from contextlib import contextmanager

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from human_eval.data import read_problems, write_jsonl

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.6.0+cu124
CUDA available: True
GPU: Tesla T4


## 2. Load Qwen2.5-Coder-1.5B-Instruct Model

In [3]:
MODEL_NAME = "Qwen/Qwen2.5-Coder-1.5B-Instruct"

# 4-bit quantization config for memory efficiency
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

print(f"Loading model: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
)
model.eval()
print("Model loaded successfully!")

Loading model: Qwen/Qwen2.5-Coder-1.5B-Instruct


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Model loaded successfully!


## 3. Load HumanEval Dataset

In [4]:
# Load HumanEval problems
problems = read_problems()
print(f"Loaded {len(problems)} HumanEval problems")

# Display sample problem
sample_id = "HumanEval/0"
print(f"\nSample Problem ({sample_id}):")
print("="*50)
print(problems[sample_id]["prompt"][:500])

Loaded 164 HumanEval problems

Sample Problem (HumanEval/0):
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



## 4. Error-Aware Prompt Template & Generation Functions

Using Qwen2.5-Coder chat template format for code completion with ERROR-AWARE prompting.

In [5]:
def create_prompt(problem_prompt: str) -> str:
    """
    Create the ERROR-AWARE prompt for Qwen/Qwen2.5-Coder-1.5B-Instruct.
    Uses the chat template format appropriate for this model.
    """
    instruction = f"""Complete the following Python function. Only output the function body code that should be added after the function signature. Do not include the function signature or any explanations.

CRITICAL REQUIREMENTS:
1. Output ONLY valid Python code wrapped in ```python and ``` markers
2. Use proper indentation (4 spaces per level) for all nested structures
3. Do NOT include the function signature - only the body
4. Do NOT include any explanations or comments outside the code block
5. Ensure all control structures (if/for/while) have properly indented bodies

{problem_prompt}"""
    
    # Qwen2.5-Coder uses ChatML format
    messages = [
        {"role": "system", "content": "You are a helpful coding assistant. Complete the given Python function accurately. Only provide the function body code wrapped in ```python code blocks, no explanations. Ensure proper indentation for all nested structures."},
        {"role": "user", "content": instruction}
    ]
    
    formatted_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    return formatted_prompt


def generate_completion(prompt: str, max_new_tokens: int = 512) -> str:
    """
    Generate code completion using Qwen/Qwen2.5-Coder-1.5B-Instruct.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,  # Greedy decoding for pass@1
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            temperature=None,
            top_p=None,
        )
    
    # Decode only the generated part
    generated_tokens = outputs[0][inputs['input_ids'].shape[1]:]
    completion = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    return completion


def extract_function_body(completion: str, prompt: str) -> str:
    """
    Extract and clean the function body from the completion.
    CRITICAL: Adds proper indentation so the code becomes part of the function body.
    """
    # Remove markdown code blocks if present
    completion = re.sub(r'```python\n?', '', completion)
    completion = re.sub(r'```\n?', '', completion)
    
    # Stop at common end markers
    stop_sequences = ["\n\n\n", "\n###", "\nif __name__", "\n# Test", "\n# Example", "\n\n#"]
    for stop in stop_sequences:
        if stop in completion:
            completion = completion[:completion.index(stop)]
    
    completion = completion.strip()
    
    # CRITICAL FIX: Add proper indentation to each line
    # The model outputs code without indentation, but it needs to be inside the function
    lines = completion.split('\n')
    indented_lines = []
    for line in lines:
        # If line is not empty and doesn't already have proper indentation, add 4 spaces
        if line.strip():  # Non-empty line
            # Check if line already starts with spaces (model sometimes includes indentation)
            if line.startswith('    '):  # Already indented
                indented_lines.append(line)
            else:
                indented_lines.append('    ' + line)
        else:  # Empty line
            indented_lines.append('')
    
    return '\n'.join(indented_lines)


print("Prompt generation functions defined.")

Prompt generation functions defined.


## 5. Execution & Error Classification

In [6]:
class TimeoutException(Exception):
    pass


@contextmanager
def time_limit(seconds: int):
    """
    Context manager to limit execution time.
    """
    def signal_handler(signum, frame):
        raise TimeoutException("Execution timed out!")
    
    old_handler = signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    
    try:
        yield
    finally:
        signal.alarm(0)
        signal.signal(signal.SIGALRM, old_handler)


def classify_error(code: str, test_code: str, entry_point: str) -> Tuple[str, str]:
    """
    Execute code and classify any errors.
    Returns: (status, error_message)
    Status can be: 'pass', 'syntax_error', 'runtime_error', 'incomplete_code', 'timeout'
    """
    # First check for syntax errors
    try:
        ast.parse(code)
    except SyntaxError as e:
        return 'syntax_error', str(e)
    
    # Check if the entry point function exists in code
    if f"def {entry_point}" not in code:
        return 'incomplete_code', f"Function '{entry_point}' not found in generated code"
    
    # Try to execute with tests
    full_code = code + "\n" + test_code
    
    try:
        ast.parse(full_code)
    except SyntaxError as e:
        return 'syntax_error', str(e)
    
    # Execute with timeout
    try:
        exec_globals = {}
        with time_limit(5):
            exec(full_code, exec_globals)
        return 'pass', ''
    except TimeoutException:
        return 'timeout', 'Execution timed out (>5 seconds)'
    except AssertionError as e:
        return 'runtime_error', f"Assertion failed: {str(e)}"
    except Exception as e:
        error_type = type(e).__name__
        return 'runtime_error', f"{error_type}: {str(e)}"


print("Execution and error classification functions defined.")

Execution and error classification functions defined.


## 6. Run Evaluation on All 164 Problems

In [7]:
# Initialize tracking variables
results = []
error_counts = defaultdict(int)
successful_problems = []
failed_problems = defaultdict(list)

# Output log file for prompts and generated outputs
output_log_path = "output_log_qwen25_coder_1.5b_error_aware_all.txt"

print(f"Starting Qwen/Qwen2.5-Coder-1.5B-Instruct evaluation on {len(problems)} problems...")
print("="*60)

with open(output_log_path, 'w', encoding='utf-8') as log_file:
    log_file.write("HumanEval Evaluation Log - Qwen/Qwen2.5-Coder-1.5B-Instruct (Error-Aware Prompt)\n")
    log_file.write(f"Model: {MODEL_NAME}\n")
    log_file.write("="*60 + "\n\n")
    
    for i, (task_id, problem) in enumerate(problems.items()):
        problem_num = int(task_id.split("/")[1])
        prompt = problem["prompt"]
        test_code = problem["test"]
        entry_point = problem["entry_point"]
        
        # Create prompt and generate completion
        formatted_prompt = create_prompt(prompt)
        completion = generate_completion(formatted_prompt)
        cleaned_completion = extract_function_body(completion, prompt)
        
        # Combine prompt with completion for full code
        # The prompt ends with the function signature, cleaned_completion is now properly indented
        full_code = prompt + "\n" + cleaned_completion
        
        # Classify result
        status, error_msg = classify_error(full_code, test_code, entry_point)
        
        # Track results
        error_counts[status] += 1
        
        if status == 'pass':
            successful_problems.append(problem_num)
        else:
            failed_problems[status].append(problem_num)
        
        results.append({
            "task_id": task_id,
            "problem_num": problem_num,
            "status": status,
            "error": error_msg,
            "completion": cleaned_completion,
            "prompt_type": "error_aware"
        })
        
        # Write to log file
        log_file.write(f"{'='*60}\n")
        log_file.write(f"PROBLEM {problem_num}: {task_id}\n")
        log_file.write(f"STATUS: {status.upper()}\n")
        log_file.write(f"{'='*60}\n\n")
        log_file.write(f"--- ORIGINAL HUMANEVAL PROMPT ---\n")
        log_file.write(f"{prompt}\n\n")
        log_file.write(f"--- FORMATTED PROMPT TO MODEL ---\n")
        log_file.write(f"{formatted_prompt}\n\n")
        log_file.write(f"--- RAW MODEL OUTPUT ---\n")
        log_file.write(f"{completion}\n\n")
        log_file.write(f"--- CLEANED COMPLETION (WITH INDENTATION) ---\n")
        log_file.write(f"{cleaned_completion}\n\n")
        log_file.write(f"--- FULL GENERATED CODE ---\n")
        log_file.write(f"{full_code}\n\n")
        if error_msg:
            log_file.write(f"--- ERROR MESSAGE ---\n")
            log_file.write(f"{error_msg}\n\n")
        log_file.write("\n")
        
        # Progress update
        if (i + 1) % 10 == 0 or (i + 1) == len(problems):
            print(f"Progress: {i+1}/{len(problems)} | Current pass rate: {error_counts['pass']/(i+1)*100:.1f}%")

print("\nEvaluation complete!")
print(f"Output log saved to: {output_log_path}")

The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Starting Qwen/Qwen2.5-Coder-1.5B-Instruct evaluation on 164 problems...


The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=

Progress: 10/164 | Current pass rate: 40.0%


The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=

...

Progress: 150/164 | Current pass rate: 54.7%


The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=

Progress: 160/164 | Current pass rate: 54.4%


The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Progress: 164/164 | Current pass rate: 53.0%

Evaluation complete!
Output log saved to: output_log_qwen25_coder_1.5b_error_aware_all.txt


## 7. Calculate & Display Results

In [8]:
# Calculate metrics
total_problems = len(problems)
passed = error_counts['pass']
syntax_errors = error_counts['syntax_error']
runtime_errors = error_counts['runtime_error']
incomplete_errors = error_counts['incomplete_code']
timeout_errors = error_counts['timeout']

pass_at_1 = (passed / total_problems) * 100
syntax_error_pct = (syntax_errors / total_problems) * 100
runtime_error_pct = (runtime_errors / total_problems) * 100
incomplete_error_pct = (incomplete_errors / total_problems) * 100
timeout_error_pct = (timeout_errors / total_problems) * 100

print("\n" + "="*60)
print("HUMANEVAL EVALUATION RESULTS - Qwen2.5-Coder-1.5B-Instruct")
print(f"Model: {MODEL_NAME}")
print("="*60)

print(f"\n PASS@1 METRIC:")
print(f"   Pass@1: {pass_at_1:.2f}% ({passed}/{total_problems})")

print(f"\n ERROR BREAKDOWN (on {total_problems} problems):")
print(f"    Passed:           {passed:3d} ({pass_at_1:.2f}%)")
print(f"    Syntax Errors:    {syntax_errors:3d} ({syntax_error_pct:.2f}%)")
print(f"    Runtime Errors:   {runtime_errors:3d} ({runtime_error_pct:.2f}%)")
print(f"    Incomplete Code:  {incomplete_errors:3d} ({incomplete_error_pct:.2f}%)")
print(f"    Timeouts:         {timeout_errors:3d} ({timeout_error_pct:.2f}%)")


HUMANEVAL EVALUATION RESULTS - Qwen2.5-Coder-1.5B-Instruct
Model: Qwen/Qwen2.5-Coder-1.5B-Instruct

 PASS@1 METRIC:
   Pass@1: 53.05% (87/164)

 ERROR BREAKDOWN (on 164 problems):
    Passed:            87 (53.05%)
    Syntax Errors:     77 (46.95%)
    Runtime Errors:     0 (0.00%)
    Incomplete Code:    0 (0.00%)
    Timeouts:           0 (0.00%)


In [9]:
# Display successful problem numbers
print("\n" + "="*60)
print("SUCCESSFULLY EXECUTED PROBLEMS")
print("="*60)

successful_problems_sorted = sorted(successful_problems)
print(f"\nTotal: {len(successful_problems_sorted)} problems passed")
print(f"\nProblem Numbers: {successful_problems_sorted}")


SUCCESSFULLY EXECUTED PROBLEMS

Total: 87 problems passed

Problem Numbers: [2, 3, 4, 7, 10, 11, 12, 13, 15, 16, 19, 22, 23, 27, 29, 30, 31, 32, 33, 34, 35, 37, 38, 41, 42, 44, 45, 48, 49, 50, 51, 52, 53, 54, 55, 57, 60, 62, 67, 70, 72, 74, 76, 77, 81, 83, 85, 88, 89, 90, 92, 95, 97, 101, 103, 104, 105, 106, 107, 108, 110, 113, 114, 116, 117, 119, 120, 121, 122, 123, 126, 128, 130, 131, 134, 138, 139, 140, 144, 145, 146, 149, 150, 151, 152, 157, 159]


In [10]:
# Display failed problems by category
print("\n" + "="*60)
print("FAILED PROBLEMS BY CATEGORY")
print("="*60)

if failed_problems['syntax_error']:
    print(f"\n Syntax Errors ({len(failed_problems['syntax_error'])}):") 
    print(f"   {sorted(failed_problems['syntax_error'])}")

if failed_problems['runtime_error']:
    print(f"\n Runtime Errors ({len(failed_problems['runtime_error'])}):") 
    print(f"   {sorted(failed_problems['runtime_error'])}")

if failed_problems['incomplete_code']:
    print(f"\n Incomplete Code ({len(failed_problems['incomplete_code'])}):") 
    print(f"   {sorted(failed_problems['incomplete_code'])}")

if failed_problems['timeout']:
    print(f"\n  Timeouts ({len(failed_problems['timeout'])}):") 
    print(f"   {sorted(failed_problems['timeout'])}")


FAILED PROBLEMS BY CATEGORY

 Syntax Errors (77):
   [0, 1, 5, 6, 8, 9, 14, 17, 18, 20, 21, 24, 25, 26, 28, 36, 39, 40, 43, 46, 47, 56, 58, 59, 61, 63, 64, 65, 66, 68, 69, 71, 73, 75, 78, 79, 80, 82, 84, 86, 87, 91, 93, 94, 96, 98, 99, 100, 102, 109, 111, 112, 115, 118, 124, 125, 127, 129, 132, 133, 135, 136, 137, 141, 142, 143, 147, 148, 153, 154, 155, 156, 158, 160, 161, 162, 163]


In [11]:
# Save summary
summary_path = "evaluation_summary_qwen25_coder_1.5b_error_aware_all.txt"

with open(summary_path, 'w') as f:
    f.write("HUMANEVAL EVALUATION SUMMARY - Qwen2.5-Coder-1.5B-Instruct\n")
    f.write(f"Model: {MODEL_NAME}\n")
    f.write("="*60 + "\n\n")
    
    f.write(f"PASS@1: {pass_at_1:.2f}% ({passed}/{total_problems})\n\n")
    
    f.write("ERROR BREAKDOWN:\n")
    f.write(f"  - Passed:          {passed} ({pass_at_1:.2f}%)\n")
    f.write(f"  - Syntax Errors:   {syntax_errors} ({syntax_error_pct:.2f}%)\n")
    f.write(f"  - Runtime Errors:  {runtime_errors} ({runtime_error_pct:.2f}%)\n")
    f.write(f"  - Incomplete Code: {incomplete_errors} ({incomplete_error_pct:.2f}%)\n")
    f.write(f"  - Timeouts:        {timeout_errors} ({timeout_error_pct:.2f}%)\n\n")
    
    f.write("SUCCESSFUL PROBLEM NUMBERS:\n")
    f.write(f"{successful_problems_sorted}\n\n")
    
    f.write("FAILED PROBLEMS BY CATEGORY:\n")
    if failed_problems['syntax_error']:
        f.write(f"  Syntax Errors: {sorted(failed_problems['syntax_error'])}\n")
    if failed_problems['runtime_error']:
        f.write(f"  Runtime Errors: {sorted(failed_problems['runtime_error'])}\n")
    if failed_problems['incomplete_code']:
        f.write(f"  Incomplete Code: {sorted(failed_problems['incomplete_code'])}\n")
    if failed_problems['timeout']:
        f.write(f"  Timeouts: {sorted(failed_problems['timeout'])}\n")

print(f"\n Summary saved to: {summary_path}")


 Summary saved to: evaluation_summary_qwen25_coder_1.5b_error_aware_all.txt


In [12]:
# Save detailed results as JSON
results_path = "evaluation_results_qwen25_coder_1.5b_error_aware_all.json"

detailed_results = {
    "model": MODEL_NAME,
    "prompt_type": "error_aware",
    "total_problems": total_problems,
    "pass_at_1": pass_at_1,
    "error_counts": dict(error_counts),
    "error_percentages": {
        "pass": pass_at_1,
        "syntax_error": syntax_error_pct,
        "runtime_error": runtime_error_pct,
        "incomplete_code": incomplete_error_pct,
        "timeout": timeout_error_pct
    },
    "successful_problems": successful_problems_sorted,
    "failed_problems": {k: sorted(v) for k, v in failed_problems.items()},
    "detailed_results": results
}

with open(results_path, 'w') as f:
    json.dump(detailed_results, f, indent=2)

print(f" Detailed results saved to: {results_path}")

 Detailed results saved to: evaluation_results_qwen25_coder_1.5b_error_aware_all.json


## 8. Final Summary

In [13]:
print("\nEVALUATION COMPLETE!")

print(f"""
Qwen/Qwen2.5-Coder-1.5B-Instruct EVALUATION (error aware) RESULTS
Model: Qwen/Qwen2.5-Coder-1.5B-Instruct
Benchmark: HumanEval ({total_problems} problems)

PASS@1: {pass_at_1:>6.2f}%

Error Breakdown:
  Passed:          {passed:>3d} problems ({pass_at_1:>5.2f}%)
  Syntax Errors:   {syntax_errors:>3d} problems ({syntax_error_pct:>5.2f}%)
  Runtime Errors:  {runtime_errors:>3d} problems ({runtime_error_pct:>5.2f}%)
  Incomplete Code: {incomplete_errors:>3d} problems ({incomplete_error_pct:>5.2f}%)
  Timeouts:        {timeout_errors:>3d} problems ({timeout_error_pct:>5.2f}%)

Output Files:
  output_log_error_aware_qwen25_coder_1.5b.txt
  evaluation_summary_error_aware_qwen25_coder_1.5b.txt
  evaluation_results_error_aware_qwen25_coder_1.5b.json
""")


EVALUATION COMPLETE!

Qwen/Qwen2.5-Coder-1.5B-Instruct EVALUATION (error aware) RESULTS
Model: Qwen/Qwen2.5-Coder-1.5B-Instruct
Benchmark: HumanEval (164 problems)

PASS@1:  53.05%

Error Breakdown:
  Passed:           87 problems (53.05%)
  Syntax Errors:    77 problems (46.95%)
  Runtime Errors:    0 problems ( 0.00%)
  Incomplete Code:   0 problems ( 0.00%)
  Timeouts:          0 problems ( 0.00%)

Output Files:
  output_log_error_aware_qwen25_coder_1.5b.txt
  evaluation_summary_error_aware_qwen25_coder_1.5b.txt
  evaluation_results_error_aware_qwen25_coder_1.5b.json

